In [1]:
import pandas as pd
import json
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import FastText
from epochlogger import EpochLogger
from sentencegenerator import SentenceGenerator

C:\Users\ML\Anaconda3\envs\gabe\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data = pd.read_csv('../data/mfp-diaries-shuffled-train.tsv',sep='\t',header=None,chunksize=40000)

### Construct the corpus with each sentence corresponding to a diary entry

In [5]:
# i = 0
# with open("../data/sentences.txt", "a",encoding='utf-8') as sentences:
#     for chunk in data:
#         for row in chunk.iterrows():
#             ex_diary = json.loads(row[1][2])
#             for meal in ex_diary:
#                 dish_list = []
#                 for dish in meal['dishes']:
#                     if dish['name'][0:20] == 'Quick Added Calories':
#                         provider = 'Quick Added Calories'
#                         item = 'Quick Added Calories'
#                     else:
#                         name, quant = dish['name'].split(',',maxsplit=1)
#                         try:
#                             provider, item = name.split(' - ')[0:2]
#                         except ValueError:
#                             if len(name.split(' - ')) ==1:
#                                 provider = name
#                                 item = ""
#                     word = '_'.join(provider.split()) + '___' + '_'.join(item.split())
#                     dish_list.append(word)
#                 sentence = " ".join(dish_list)
#                 sentences.write(sentence+'\n')
#             i += 1
#             print(i,end='\r')

### Construct the corpus with each sentence corresponding to an individual food name

In [6]:
i = 0
with open("../data/foods_as_sentences_train.txt", "a",encoding='utf-8') as sentences:
    for chunk in data:
        for row in chunk.iterrows():
            ex_diary = json.loads(row[1][2])
            for meal in ex_diary:
                for dish in meal['dishes']:
                    if dish['name'][0:20] == 'Quick Added Calories':
                        provider = 'Quick Added Calories'
                        item = 'Quick Added Calories'
                    else:
                        name, quant = dish['name'].split(',',maxsplit=1)
                        try:
                            provider, item = name.split(' - ')[0:2]
                        except ValueError:
                            if len(name.split(' - ')) ==1:
                                provider = name
                                item = ""
                    sentence = ' '.join(provider.split()) + ' ' + ' '.join(item.split())
                    sentences.write(sentence+'\n')
            i += 1
            print(i,end='\r')

In [7]:
sentences.close()

In [8]:
total_ex = i

## Train Word2Vec

In [10]:
epoch_logger = EpochLogger()
sentences = SentenceGenerator('../data/foods_as_sentences_train.txt')
model = Word2Vec(sentences,size=100,window=6,min_count=10)
model.train(sentences, total_examples=total_ex, epochs=10,callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end


(205962286, 226268090)

In [11]:
model.save("../word2vecmodels/model_foods_as_sentences_train_only.model")

In [13]:
model.wv.index2word

['Cheese',
 'Chicken',
 'Raw',
 'Generic',
 'Milk',
 '&',
 'Coffee',
 'Chocolate',
 'Butter',
 'and',
 'With',
 'Bread',
 'Homemade',
 'Protein',
 'Egg',
 'Organic',
 'Cream',
 'Whole',
 'Light',
 'White',
 'Fresh',
 'Free',
 'Sugar',
 'Peanut',
 'Salad',
 'Yogurt',
 'Bar',
 'Breast',
 'Almond',
 'Sweet',
 'Fat',
 'Rice',
 'Eggs',
 'Natural',
 'Turkey',
 'Apple',
 'Vanilla',
 'Greek',
 'Green',
 'Oil',
 'Banana',
 'Value',
 'Red',
 'Calories',
 'Quick',
 'Great',
 'Tesco',
 'Original',
 'Added',
 'Wheat',
 'Fruit',
 'Kroger',
 'Honey',
 'Trader',
 'Brewed',
 '100%',
 'Sauce',
 'Beef',
 'Large',
 'Tea',
 'Kraft',
 'Bacon',
 'Cheddar',
 'Potato',
 'from',
 'grounds',
 'Spinach',
 'Chips',
 'Roasted',
 'Creamer',
 'Medium',
 'Brown',
 'Black',
 'Farms',
 'Dressing',
 'Tomato',
 'Half',
 'Beans',
 'Oats',
 'Frozen',
 'Soup',
 'Baby',
 'Coconut',
 'the',
 'Whey',
 'Mix',
 'Broccoli',
 'French',
 'Olive',
 'Pizza',
 'Baked',
 'Kirkland',
 '1',
 'Sausage',
 'Oatmeal',
 'Carrots',
 'Dark',
 'P

In [11]:
model.wv.most_similar("Raspberries")

[('(Raspberries', 0.9843159914016724),
 ('Rasberries', 0.9639889597892761),
 ('Raspberriess', 0.9560129642486572),
 ('raspberries', 0.951819658279419),
 ('Fruit/Raspberries', 0.9398398995399475),
 ('Bluberries', 0.9139211177825928),
 ('Straberries', 0.9107884168624878),
 ('Mulberries', 0.9078725576400757),
 ('Strwberries', 0.8971955180168152),
 ('bluberries', 0.8908132314682007)]

In [15]:
model.wv.most_similar('Coffee')

[('Coffe', 0.6892114877700806),
 ('Cappuccino', 0.6448010206222534),
 ('Tea', 0.6218572854995728),
 ('coffee', 0.5886411070823669),
 ('Decaf', 0.5647045373916626),
 ('Americano', 0.563005805015564),
 ('Latte', 0.5549725294113159),
 ('Coffee-', 0.5541614294052124),
 ('Folgers', 0.5238036513328552),
 ('Espresso', 0.5066592693328857)]

In [16]:
model.wv.most_similar('Beer')

[('Lager', 0.682033360004425),
 ('Nastro', 0.6280746459960938),
 ('Ale', 0.6231441497802734),
 ('Brewery', 0.5951564311981201),
 ('Shiner', 0.5871891975402832),
 ('Beer:', 0.5701658725738525),
 ('Rootbeer', 0.5633116960525513),
 ('Azzurro', 0.5631240606307983),
 ('Azzuro', 0.5569137334823608),
 ('Ipa', 0.5559419989585876)]

### Train FastText

In [3]:
# epoch_logger = EpochLogger()
sentences = SentenceGenerator('../data/foods_as_sentences_train.txt')
model = FastText(sentences,min_count=10)

In [8]:
# fid = open("../fasttextmodels/model_foods_as_sentences_train_only_fasttext.model","w")
model.save("../fasttextmodels/model_foods_as_sentences_train_only_fasttext.model")

In [9]:
model.wv.index2word

['Cheese',
 'Chicken',
 'Raw',
 'Generic',
 'Milk',
 '&',
 'Coffee',
 'Chocolate',
 'Butter',
 'and',
 'With',
 'Bread',
 'Homemade',
 'Protein',
 'Egg',
 'Organic',
 'Cream',
 'Whole',
 'Light',
 'White',
 'Fresh',
 'Free',
 'Sugar',
 'Peanut',
 'Salad',
 'Yogurt',
 'Bar',
 'Breast',
 'Almond',
 'Sweet',
 'Fat',
 'Rice',
 'Eggs',
 'Natural',
 'Turkey',
 'Apple',
 'Vanilla',
 'Greek',
 'Green',
 'Oil',
 'Banana',
 'Value',
 'Red',
 'Calories',
 'Quick',
 'Great',
 'Tesco',
 'Original',
 'Added',
 'Wheat',
 'Fruit',
 'Kroger',
 'Honey',
 'Trader',
 'Brewed',
 '100%',
 'Sauce',
 'Beef',
 'Large',
 'Tea',
 'Kraft',
 'Bacon',
 'Cheddar',
 'Potato',
 'from',
 'grounds',
 'Spinach',
 'Chips',
 'Roasted',
 'Creamer',
 'Medium',
 'Brown',
 'Black',
 'Farms',
 'Dressing',
 'Tomato',
 'Half',
 'Beans',
 'Oats',
 'Frozen',
 'Soup',
 'Baby',
 'Coconut',
 'the',
 'Whey',
 'Mix',
 'Broccoli',
 'French',
 'Olive',
 'Pizza',
 'Baked',
 'Kirkland',
 '1',
 'Sausage',
 'Oatmeal',
 'Carrots',
 'Dark',
 'P

In [10]:
model.wv.most_similar('Raspberries')

[('(Raspberries', 0.9843159914016724),
 ('Rasberries', 0.9639889597892761),
 ('Raspberriess', 0.9560129642486572),
 ('raspberries', 0.951819658279419),
 ('Fruit/Raspberries', 0.9398398995399475),
 ('Bluberries', 0.9139211177825928),
 ('Straberries', 0.9107884168624878),
 ('Mulberries', 0.9078725576400757),
 ('Strwberries', 0.8971955180168152),
 ('bluberries', 0.8908132314682007)]

In [12]:
model.wv.most_similar('Coffee')

[('Coffee®', 0.9459692239761353),
 ('*Coffee', 0.9453116655349731),
 ('Coffee/Tea', 0.9430606961250305),
 ('Coffee;', 0.9426363706588745),
 ('Coffee.', 0.9417710304260254),
 ('Coffee*', 0.9390425682067871),
 ('Coffee(2', 0.9317131042480469),
 ('Tea/Coffee', 0.9281260967254639),
 ('"Coffee', 0.9252644777297974),
 ('Coffe', 0.919734537601471)]

In [13]:
model.wv.most_similar('Beer')

[('Beer:', 0.9087232947349548),
 ('Beer-', 0.8899614810943604),
 ('Erdbeer', 0.7212141752243042),
 ('Himbeer', 0.718732476234436),
 ('Lager', 0.7056966423988342),
 ('Beer)', 0.7017104625701904),
 ('Ale', 0.6995254755020142),
 ('deer', 0.6960493326187134),
 ('beer', 0.6931201815605164),
 ('Beer-battered', 0.6906496286392212)]

In [17]:
oov_word = "fishhh" 
oov_word in model.wv.vocab

False

In [18]:
oov_vec = model.wv[oov_word] 

In [22]:
model.wv.similar_by_vector(oov_vec)

'fish'